In [90]:
from llama_model import load_model, generate_text
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
from groq import Groq
import json
import os

os.environ["GROQ_API_KEY"] = "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"


In [83]:

tokenizer, model = load_model()
rag_model = SentenceTransformer('all-MiniLM-L6-v2') # all-mpnet-base-v2

In [84]:
val_df = pd.read_json("val.json", lines=True)
val_df.head()

,id,paragraphs,question,question_decomposition,answer,answer_aliases,answerable
0,2hop__460946_294723,"[{'idx': 0, 'title': 'Grant's First Stand', 'p...",Who is the spouse of the Green performer?,"[{'id': 460946, 'question': 'Green >> performe...",Miquette Giraudy,[],True
1,2hop__252311_366220,"[{'idx': 0, 'title': 'SICRAL 1B', 'paragraph_t...",Who founded the company that distributed the f...,"[{'id': 252311, 'question': 'UHF >> distribute...",Mike Medavoy,[],True
2,2hop__701895_752697,"[{'idx': 0, 'title': 'Pangi Territory', 'parag...",What administrative territorial entity is the ...,"[{'id': 701895, 'question': 'Ciudad Deportiva ...",Tamaulipas,[],True
3,2hop__259228_793698,"[{'idx': 0, 'title': 'Richmond, Virginia', 'pa...",Where is Ulrich Walter's employer headquartered?,"[{'id': 259228, 'question': 'Ulrich Walter >> ...",Cologne,[],True
4,2hop__481349_302087,"[{'idx': 0, 'title': 'Market Square (High Poin...",Which company owns the manufacturer of Learjet...,"[{'id': 481349, 'question': 'Learjet 60 >> man...",Bombardier Inc.,[Bombardier],True


In [85]:
val_df.shape

(2417, 7)

In [86]:
val_df['question_decomposition'][0]

[{'id': 460946,
  'question': 'Green >> performer',
  'answer': 'Steve Hillage',
  'paragraph_support_idx': 10},
 {'id': 294723,
  'question': '#1 >> spouse',
  'answer': 'Miquette Giraudy',
  'paragraph_support_idx': 5}]

In [87]:
# implement RAG to find the closest paragraphs to the question
def get_closest_paragraphs(question, paragraphs, top_k=5, second_closest=False):
    # print("<<<<<", paragraphs)
    question_embedding = rag_model.encode(question)
    paragraph_embeddings = [rag_model.encode(paragraph) for paragraph in paragraphs] 
    similarity = [np.dot(question_embedding, paragraph_embedding) / (np.linalg.norm(question_embedding) * np.linalg.norm(paragraph_embedding)) for paragraph_embedding in paragraph_embeddings]
    
    # return the topk paragraphs
    if second_closest:
        topk_idx = np.argsort(similarity)[-top_k-1:-1].tolist()
    else:
        topk_idx = np.argsort(similarity)[-top_k:].tolist() # in this line, we are sorting the similarity scores and getting the topk indexes
    topk_sentences = [paragraphs[i] for i in topk_idx]
    # print(topk_idx)
    return topk_sentences

In [88]:
# client = Groq()

# chat_completion = client.chat.completions.create(

#     messages=[
#         {
#             "role": "system",
#             "content": "you are a helpful assistant."
#         },
#         # Set a user message for the assistant to respond to.
#         {
#             "role": "user",
#             "content": "Explain the importance of fast language models",
#         }
#     ],

#     # The language model which will generate the completion.
#     model="llama-3.2-3b-preview",
#     temperature=0.5,
#     max_tokens=8192,
#     top_p=1,

#     # A stop sequence is a predefined or user-specified text string that
#     # signals an AI to stop generating content, ensuring its responses
#     # remain focused and concise. Examples include punctuation marks and
#     # markers like "[end]".
#     stop=None,

#     # If set, partial message deltas will be sent.
#     stream=False,
# )

# # Print the completion returned by the LLM.
# print(chat_completion.choices[0].message.content)

In [94]:
client = Groq()

use_iterative_RAG = True

print('Reponse\t\tGround Truth')

outputs = []
responses = []
gts = []
for i in range(3000):
    
    with open('RAG_results.json', 'r') as f:
        add_to_json = json.load(f)
        
    if add_to_json.get(f'{i}'):
        continue
    
    paragraphs = val_df["paragraphs"][i]
    paragraphs = [p['paragraph_text'] for p in paragraphs]
    
            
    complex_question = val_df["question"][i]
    # print(complex_question)
    
    closest_paragraphs = get_closest_paragraphs(complex_question, paragraphs, top_k=3)
    if use_iterative_RAG:
        second_closest_paragraphs = [get_closest_paragraphs(cp, paragraphs, top_k=2, second_closest=True) for cp in closest_paragraphs]
        second_closest_paragraphs = [item for sublist in second_closest_paragraphs for item in sublist]
        # print(">>>", second_closest_paragraphs)
        closest_paragraphs.extend(second_closest_paragraphs)
        closest_paragraphs = list(set(closest_paragraphs))
    
    paragraphs = " ".join(closest_paragraphs)
    
    gt_para_idxs = val_df["question_decomposition"][i]
    gt_para_idxs = [gt['paragraph_support_idx'] for gt in gt_para_idxs]
    # print("gt_para_idxs : ", gt_para_idxs)
    
    if len(paragraphs) > 4000:
        paragraphs = paragraphs[:4000]    
    
    # prompt = f'''CONTEXT : {paragraphs}.\nQUESTION : '{complex_question}'.\nProvide the correct answer to the question in the following format: "answer: (answer under 5 words)"'''
    # print(prompt)
    # response = generate_text(tokenizer, model, prompt, max_new_tokens=10)
    # model_answer = response[len(prompt):-1].strip()
    

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "Using the CONTEXT, give the most accurate answer to the QUESTION in the format `ANSWER : (upto 5 word answer)"
            },
            {
                "role": "user",
                "content": f"CONTEXT : {paragraphs} \nQUESTION : '{complex_question}'. ANSWER :",
            }
        ],

        # The language model which will generate the completion.
        model="llama-3.1-8b-instant",
        temperature=0.4,
        max_tokens=10,
        top_p=1,
        stop=None,
        stream=False,
    )

    # Print the completion returned by the LLM.
    model_answer = chat_completion.choices[0].message.content
    
    model_answer = model_answer.replace("ANSWER : ", "")
    model_answer = model_answer.replace("\n", "")
    model_answer = model_answer.replace(".", "")
    model_answer = model_answer.strip()
    
    gt = val_df["answer"][i].replace(".", "")
    
    
    add_to_json.update({
        i: {
            'model_answer': model_answer,
            'gt': gt
        }
    })
    
    with open('RAG_results.json', 'w') as f:
        json.dump(add_to_json, f, indent=4)
    
    
    print(f'{model_answer}\t\t{gt}')
    
    # outputs.append(response)
    responses.append(model_answer)
    gts.append(gt)

Reponse		Ground Truth
Not specified in context		Miquette Giraudy
Richard O'Brien		Mike Medavoy
The municipality of Nuevo Laredo		Tamaulipas
I couldn't find information		Cologne
Bombardier Inc owns Bomb		Bombardier Inc
Daniel Webster's child		Fletcher Webster
Anne of Austria		Marie de' Medici
Promoting European integration		European integration
Adelphia Communications failed		Time Warner Cable
Cabo Delgado Province		Cabo Delgado Province
Benevento Calcio plays Serie		Lega Pro Prima Divisione
'The Celestine Prophecy		Anderson Tapes
I couldn't find information about Call		Knowsley
Pecos County shares border with		Crockett County
Holley, New York County		Orleans County
No specific award mentioned		Academy Award for Best Animated Short Film
Blair International Baccalaureate		Blair High School
Nicole Kidman's spouse		Keith Urban
King Gustav Vasa of Sweden		Svante Stensson Sture
Randall County, Texas		Randall County
Universal Music Japan		Derrty Entertainment
Not mentioned in the context		Exe

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01jczqxp2nf089qng21e2vba75` on : Limit 500000, Used 499497, Requested 1034. Please try again in 1m31.608399999s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}

In [ ]:
# save the gts
with open("iterative_RAG_outputs.txt", "w") as f:
    for response in responses:
        f.write(response + "\n")
        
with open("iterative_RAG_gts.txt", "w") as f:
    for gt in gts:
        f.write(gt + "\n")


In [ ]:
gts, responses

(['Miquette Giraudy',
  'Mike Medavoy',
  'Tamaulipas',
  'Cologne',
  'Bombardier Inc',
  'Fletcher Webster',
  "Marie de' Medici",
  'European integration',
  'Time Warner Cable',
  'Cabo Delgado Province',
  'Lega Pro Prima Divisione',
  'Anderson Tapes',
  'Knowsley',
  'Crockett County',
  'Orleans County',
  'Academy Award for Best Animated Short Film',
  'Blair High School',
  'Keith Urban',
  'Svante Stensson Sture',
  'Randall County'],
 ['No information provided about spouse',
  'Kevin Smith',
  'Municipality of Nuevo Laredo',
  'There is no mention of Ulrich Walter, so',
  'Bombardier Inc',
  'Daniel Bremer Juell',
  'Anne of Austria',
  'Promoting European integration',
  'Adelphia Communications Corporation',
  'Niassa Province',
  'Serie A',
  'The Timothy Files',
  'Merseyside',
  'Texas',
  'Holley, New York',
  'PEN/Diamonstein-Spielvog',
  'Blair International Baccalaureate School',
  'Nicole Kidman',
  'Svante Turesson Bielke',
  'Randall County, Texas'])

In [ ]:
# compare the responses and the ground truths with different metrics in NLP

# BLEU score
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score
from pycocoevalcap.cider.cider import Cider
from pycocoevalcap.spice.spice import Spice
from rouge import Rouge


rouge = Rouge()
cider = Cider()
spice = Spice()
exact_matches = 0
num_partial_matches = 0

bleu_scores = []
cider_scores = []
meteor_scores = []
rouge_scores = []
spice_scores = []
for i in range(len(gts)):
    # bleu_scores.append(sentence_bleu([gts[i].split()], responses[i].split()))
    # cider_scores.append(cider.compute_score(gts[i], responses[i]))
    # meteor_scores.append(meteor_score(gts[i], responses[i]))
    rouge_scores.append(rouge.get_scores(responses[i], gts[i]))
    # spice_scores.append(spice.compute_score(gts[i], responses[i]))
    # exact match
    if gts[i] == responses[i]:
        exact_matches += 1
        
    words_gt = gts[i].split()
    words_response = responses[i].split()
    
    # partial match by seeing how may words in the response are in the ground truth
    num_partial_matches += (len(set(words_gt).intersection(set(words_response)))
    
     
rouge_1_f = 0
rouge_2_f = 0
rouge_l_f = 0   

for i in range(len(rouge_scores)):
    rouge_1_f+= rouge_scores[i][0]['rouge-1']['f']
    rouge_2_f+= rouge_scores[i][0]['rouge-2']['f']
    rouge_l_f+= rouge_scores[i][0]['rouge-l']['f']
    
rouge_1_f = rouge_1_f/len(rouge_scores)
rouge_2_f = rouge_2_f/len(rouge_scores)
rouge_l_f = rouge_l_f/len(rouge_scores)

print("Average ROUGE-1 F1 score: ", rouge_1_f)
print("Average ROUGE-2 F1 score: ", rouge_2_f)
print("Average ROUGE-L F1 score: ", rouge_l_f)
    
        
print("Exact matches: ", exact_matches/len(gts))
print("Partial matches: ", num_partial_matches/len(gts))

# print("BLEU scores: ", bleu_scores)
# print("CIDEr scores: ", cider_scores)
# print("METEOR scores: ", meteor_scores)
# print("ROUGE scores: ", rouge_scores)
# print("SPICE scores: ", spice_scores)

Average ROUGE-1 F1 score:  0.1752380937731973
Average ROUGE-2 F1 score:  0.08333333286111111
Average ROUGE-L F1 score:  0.1752380937731973
Exact matches:  0.05
Partial matches:  0.3
